This dataset is the ENCRON mail dataset that has been split into two - ham and spam 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os

import codecs

**Reading the Contents of the Files in the Enron Folder**



The folder has two folders - ham & spam.
The lines of code below are doing the following.
* define a function that takes in a folder as a parameter 
* call the os.listdir function on the folder and save it as a file (os.litdir function gets the list of all files and directories in the specified directory)
* initiate an empty list a_list
* using the for loop to access all the files - lists of files in the folder
* if the files doesnt start with '.', open the files, append to the a_list 
* return the files as a list - a_list

In [3]:
def read_in(folder):
    files = os.listdir(folder)
    a_list = []
    for a_file in files:
        if not a_file.startswith("."):
            f = codecs.open(folder + a_file, "r", encoding = "ISO-8859-1", errors="ignore")
            a_list.append(f.read())
            f.close()
    return a_list

**Define Classes and Labels**

In [4]:
#call the read_in function on spam_list to store the folders of spam  
spam_list = read_in('/content/gdrive/MyDrive/enron1/spam/')


In [ ]:
os.chdir('/content/gdrive/MyDrive')
!ls

In [6]:
#print the first item in the list 
print(len(spam_list))
print(spam_list[0])


1500
Subject:! Ebay reguiar verification
% words% words% words% words
Dle! A; rcuelbiaiyyou: s, e: r,
% words% words% words
% words% words% words
Regards,
E: bla: y% words
% words% words% words
% words% words
% words
% words% words
% words% words



In [7]:
#call the read_in function on ham_list to store the folders of ham and print the first item in the list 
ham_list = read_in("/content/gdrive/MyDrive/enron1/ham/")
print(len(ham_list))
print(ham_list[0])


3672
Subject: fw: gop action plan: what you can do!
- - - - - original message - - - - -
From: alex fagen [mailto: alex@ fagens. Net]
Sent: wednesday, november 15, 2000 9: 56 pm
To: undisclosed - recipient:@ rad 3. 1 stup. Com;
Subject: fw: gop action plan: what you can do!
- - - - - original message - - - - -
From: kenneth r. Pybus
To:
Sent: wednesday, november 15, 2000 9: 28 am
Subject: fwd: gop action plan: what you can do!
> I got this from paul sperry at worldnetdaily
>
>
> >
> > I don' t know who joe morris is, but this sounds like a plan! (note that
> > this has been circulating since last thursday.)
> > - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
> > united republican fund of illinois
> > chicago
> >
> > november 9, 2000
> >
> > from: joseph a. Morris, president
> > to: urf leaders and friends
> >
> > herewith is some practical advice on action that every citizen can take
> > in the event that the florida recount confirms the election of geor

Now, we have the files - ham & spam stored as two lists. It is better to merge them together. A list comprehension will help to merge the two lists together. The list comprehension will retain the mails and their labels. 

Shuffle the list of data because we will eventually split them into test and train. 

Using the list comprehension, initiate a variable "allemails" that will take the spam mails and contents as well as add the ham mails. Using a random seed of 42 that will shuffle all the emails. 

In [8]:
import random 
allemails = [(email_content,"spam") for email_content in spam_list]
allemails += [(email_content,"ham") for email_content in ham_list]
random.seed(42)
random.shuffle(allemails)
print(f"Dataset size = {str(len(allemails))} emails")

Dataset size = 5172 emails


**Split Text into Words**

using the tokenizer attribute in the Natural Language Processing ToolKit

In [9]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Code Explanation**

- define a tokenize function that takes in input as a argument
- initiate an empty word_list 
- using for loop to access every word in the word_tokenize module,
- append the word to the word_list and return the word_list

In [10]:
def tokenize(input):
  word_list = []
  for word in word_tokenize(input):
    word_list.append(word)
  return word_list
input = "Iyanuoluwa is a beauty queen. She is smart"
print(tokenize(input))

['Iyanuoluwa', 'is', 'a', 'beauty', 'queen', '.', 'She', 'is', 'smart']


**Text Normalization & Features Extraction**

this steps help to normalize the text. basically, turning the text to lower case. also, you get to extract the features of the data from tokenized list.

In [11]:
def get_features(text):
  features = {}
  word_list = [word for word in word_tokenize(text.lower())]
  for word in word_list:
    features[word] = True
  return features

all_features = [(get_features(email),label)
                 for (email, label) in allemails]

              

In [12]:
print(get_features("We are buying Lotteries Today!"))

{'we': True, 'are': True, 'buying': True, 'lotteries': True, 'today': True, '!': True}


In [13]:
print(len(all_features))

5172


In [14]:
print(len(all_features[0][0]))

50


In [15]:
print(len(all_features[99][0]))

17


In [16]:
all_features[99][0]

{'subject': True,
 ':': True,
 'calpine': True,
 'daily': True,
 'gas': True,
 'nomination': True,
 'as': True,
 'we': True,
 'spoke': True,
 'of': True,
 'this': True,
 'morning': True,
 '.': True,
 '>': True,
 '-': True,
 '1.': True,
 'doc': True}

In [17]:
all_features[99]

({'subject': True,
  ':': True,
  'calpine': True,
  'daily': True,
  'gas': True,
  'nomination': True,
  'as': True,
  'we': True,
  'spoke': True,
  'of': True,
  'this': True,
  'morning': True,
  '.': True,
  '>': True,
  '-': True,
  '1.': True,
  'doc': True},
 'ham')

In [18]:
type(all_features)

list

**Training A Naive Bayes Classifier.**

This is one of the common models used for classification. With the code below, you split the data into training and test sets with a 80:20 proportion. Train the classifer with 80% of the data and evaluate i.e test whether the classifier is working accurately with the remaining 20% of the data.

Code Explanation: 
define a train function that takes in two arguements - features (remember the features are the already prepared data that's ready for processing) and proportions you want to divide the features into training and testing sets.  

In [19]:
from nltk import NaiveBayesClassifier, classify

def train(features, proportion):
    train_size = int(len(features) * proportion)
    # initialise the training and test sets
    train_set, test_set = features[:train_size], features[train_size:]
    print (f"Training set size = {str(len(train_set))} emails")
    print (f"Test set size = {str(len(test_set))} emails")
    # train the classifier
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier

train_set, test_set, classifier = train(all_features, 0.8)

Training set size = 4137 emails
Test set size = 1035 emails


**Test The Classifier**

Let's see whether the model can accurately detect whether an email is a spam or ham. How can you do that, you use metrics. One of which is **accuracy**.

In [20]:
def evaluate(train_set, test_set, classifier):
    # check how the classifier performs on the training and test sets
    print (f"Accuracy on the training set = {str(classify.accuracy(classifier, train_set))}")
    print (f"Accuracy on the test set = {str(classify.accuracy(classifier, test_set))}")    
    # check which words are most informative for the classifier
    classifier.show_most_informative_features(50)

evaluate(train_set, test_set, classifier)

Accuracy on the training set = 0.9625332366449117
Accuracy on the test set = 0.9497584541062802
Most Informative Features
               forwarded = True              ham : spam   =    201.0 : 1.0
                    2004 = True             spam : ham    =    147.0 : 1.0
            prescription = True             spam : ham    =    132.5 : 1.0
                    spam = True             spam : ham    =     90.8 : 1.0
                    2005 = True             spam : ham    =     81.1 : 1.0
                     ect = True              ham : spam   =     78.2 : 1.0
                      ex = True             spam : ham    =     73.1 : 1.0
                creative = True             spam : ham    =     71.5 : 1.0
              nomination = True              ham : spam   =     71.0 : 1.0
             medications = True             spam : ham    =     68.3 : 1.0
                   adobe = True             spam : ham    =     66.7 : 1.0
                      cc = True              ham : sp

From the above results;

Accuracy on the training set = 0.9625332366449117
Accuracy on the test set = 0.9497584541062802

That's really good. 

In [21]:
from nltk.text import Text

def concordance(data_list, search_word):
    for email in data_list:
        word_list = [word for word in word_tokenize(email.lower())]
        text_list = Text(word_list)
        if search_word in word_list:
            text_list.concordance(search_word)


print ("STOCKS in HAM:")
concordance(ham_list, "stocks")
print ("\n\nSTOCKS in SPAM:")
concordance(spam_list, "stocks")

STOCKS in HAM:
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Displaying 1 of 1 matches:
ur member directory . * follow your stocks and news headlines , exchange files
Displaying 1 of 1 matches:
ad my portfolio is diversified into stocks that have lost even more money than


STOCKS in SPAM:
Displaying 4 of 4 matches:
n this stock . some of these smal | stocks are absoiuteiy fiying , as many of 
 statements . as with many microcap stocks , todays company has additional ris
biication pertaining to investing , stocks , securities must be understood as 
ntative before deciding to trade in stocks featured within this publication . 
Displaying 3 of 3 matches:
might occur . as with many microcap stocks , today ' s company has additiona |
is emai | pertaining to investing , stocks , securities must be understood as 
ntative before 

In [22]:
test_spam_list = ["Participate in our new lottery!", "Try out this new medicine"]
test_ham_list = ["See the minutes from the last meeting attached", 
                 "Investors are coming to our office on Monday"]

test_emails = [(email_content, "spam") for email_content in test_spam_list]
test_emails += [(email_content, "ham") for email_content in test_ham_list]

new_test_set = [(get_features(email), label) for (email, label) in test_emails]

evaluate(train_set, new_test_set, classifier)

Accuracy on the training set = 0.9625332366449117
Accuracy on the test set = 1.0
Most Informative Features
               forwarded = True              ham : spam   =    201.0 : 1.0
                    2004 = True             spam : ham    =    147.0 : 1.0
            prescription = True             spam : ham    =    132.5 : 1.0
                    spam = True             spam : ham    =     90.8 : 1.0
                    2005 = True             spam : ham    =     81.1 : 1.0
                     ect = True              ham : spam   =     78.2 : 1.0
                      ex = True             spam : ham    =     73.1 : 1.0
                creative = True             spam : ham    =     71.5 : 1.0
              nomination = True              ham : spam   =     71.0 : 1.0
             medications = True             spam : ham    =     68.3 : 1.0
                   adobe = True             spam : ham    =     66.7 : 1.0
                      cc = True              ham : spam   =     65.9

Classify the new mails to test the classifier

In [23]:
for email in test_spam_list:
    print (email)
    print (classifier.classify(get_features(email)))
for email in test_ham_list:
    print (email)
    print (classifier.classify(get_features(email)))

Participate in our new lottery!
spam
Try out this new medicine
spam
See the minutes from the last meeting attached
ham
Investors are coming to our office on Monday
ham


ASSIGNMENT

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
#call the read_in function on ham_list to store the folders of ham and print the first item in the list 
new_ham_list = read_in("/content/gdrive/MyDrive/enron2/ham/")

#call the read_in function on spam_list to store the folders of spam  
new_spam_list = read_in('/content/gdrive/MyDrive/enron2/spam/')

In [27]:
import random

new_all_emails = [(email_content, "spam") for email_content in new_spam_list]
new_all_emails += [(email_content, "ham") for email_content in new_ham_list]
random.seed(42)
random.shuffle(new_all_emails)
print (f"Dataset size = {str(len(new_all_emails))} emails")

Dataset size = 5857 emails


In [28]:
import nltk
from nltk import word_tokenize

def get_features(text): 
    features = {}
    word_list = [word for word in word_tokenize(text.lower())]
    for word in word_list:
        features[word] = True
    return features

new_features = [(get_features(email), label) for (email, label) in new_all_emails]

print(get_features("Participate In Our New Lottery NOW!"))
print(len(new_features))
print(len(new_features[0][0]))
print(len(new_features[99][0]))

{'participate': True, 'in': True, 'our': True, 'new': True, 'lottery': True, 'now': True, '!': True}
5857
161
413


In [29]:
from nltk import NaiveBayesClassifier, classify

def train(features, proportion):
    train_size = int(len(features) * proportion)
    # initialise the training and test sets
    train_set, test_set = features[:train_size], features[train_size:]
    print (f"Training set size = {str(len(train_set))} emails")
    print (f"Test set size = {str(len(test_set))} emails")
    # train the classifier
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier

train_set, test_set, classifier = train(new_features, 0.8)

Training set size = 4685 emails
Test set size = 1172 emails


In [30]:
def evaluate(train_set, test_set, classifier):
    # check how the classifier performs on the training and test sets
    print (f"Accuracy on the training set = {str(classify.accuracy(classifier, train_set))}")
    print (f"Accuracy on the test set = {str(classify.accuracy(classifier, test_set))}")    
    # check which words are most informative for the classifier
    classifier.show_most_informative_features(50)

evaluate(train_set, test_set, classifier)

Accuracy on the training set = 0.9985058697972252
Accuracy on the test set = 0.9914675767918089
Most Informative Features
                   vince = True              ham : spam   =    514.0 : 1.0
                     ect = True              ham : spam   =    252.1 : 1.0
                    2005 = True             spam : ham    =    144.3 : 1.0
                      cc = True              ham : spam   =    135.6 : 1.0
                     oem = True             spam : ham    =    102.5 : 1.0
                 engines = True             spam : ham    =     98.7 : 1.0
                     doc = True              ham : spam   =     83.2 : 1.0
                   logos = True             spam : ham    =     81.4 : 1.0
           advertisement = True             spam : ham    =     78.7 : 1.0
                     php = True             spam : ham    =     77.6 : 1.0
                 artwork = True             spam : ham    =     71.8 : 1.0
               forwarded = True              ham : sp

**Combine Both Datasets Together**

In [31]:
spam_list = read_in('/content/gdrive/MyDrive/enron1/spam/') + read_in('/content/gdrive/MyDrive/enron2/spam/')
print(len(spam_list))
ham_list = read_in('/content/gdrive/MyDrive/enron1/ham/') + read_in('/content/gdrive/MyDrive/enron2/ham/')
print(len(ham_list))

all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.shuffle(test_emails)

all_features = [(get_features(email), label) for (email, label) in all_emails]
print(len(all_features))

train_set, test_set, classifier = train(all_features, 0.8)
evaluate(train_set, new_test_set, classifier)


2996
8033
11029
Training set size = 8823 emails
Test set size = 2206 emails
Accuracy on the training set = 0.9822055990026068
Accuracy on the test set = 1.0
Most Informative Features
                   meter = True              ham : spam   =    263.8 : 1.0
                   vince = True              ham : spam   =    200.2 : 1.0
                     sex = True             spam : ham    =    195.1 : 1.0
                     nom = True              ham : spam   =    194.9 : 1.0
                     php = True             spam : ham    =    182.1 : 1.0
            prescription = True             spam : ham    =    169.2 : 1.0
                     ect = True              ham : spam   =    167.5 : 1.0
                    spam = True             spam : ham    =    145.8 : 1.0
               forwarded = True              ham : spam   =    136.1 : 1.0
                     fyi = True              ham : spam   =    134.2 : 1.0
                    2005 = True             spam : ham    =    128.